Change image: {'type': 'Image', 'bands': [{'id': 'change', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 600000, 0, -10, 2500020]}]}
Significant changes count: 1


In [3]:
import ee

import geemap
# from sar_change_detection.settings import env, BASE_DIR
import os
import json
from PIL import Image
import json

# Initialize Earth Engine
service_account = "django@sih-sar-prathameshks2003.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(service_account, "service-account-key.json")
ee.Initialize(credentials)



In [4]:
def get_closest_images(s2_collection, start_date, end_date):
    """
    Gets the closest non-None Sentinel-2 images BEFORE the specified start and end dates.

    Args:
        s2_collection: An ee.ImageCollection of Sentinel-2 images.
        start_date: The start date of the image search.
        end_date: The end date of the image search.

    Returns:
        A list containing the closest non-None Sentinel-2 images BEFORE the start and end dates.
    """

    start_images = s2_collection.filterDate('1900-01-01', start_date).sort('system:time_start', False)
    end_images = s2_collection.filterDate('1900-01-01', end_date).sort('system:time_start', False)

    closest_start_image = None
    closest_end_image = None

    # Iterate to find the first non-None image for the start date
    start_list = start_images.toList(start_images.size()) # convert the collection to a list
    for i in range(start_images.size().getInfo()):
        img = ee.Image(start_list.get(i)) # get the image from the list using index i
        if img.getInfo() is not None:
            closest_start_image = img
            break

    # Iterate to find the first non-None image for the end date
    end_list = end_images.toList(end_images.size())
    for i in range(end_images.size().getInfo()):
        img = ee.Image(end_list.get(i))
        if img.getInfo() is not None:
            closest_end_image = img
            break

    return [closest_start_image, closest_end_image]

In [52]:

def detect_changes():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-07-01"
            end_date = "2024-09-28"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            # Load Sentinel-2 imagery
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get start and end images
            s2_start = s2_collection.filterDate(start_date, ee.Date(start_date).advance(1, 'month')).median()
            s2_end = s2_collection.filterDate(ee.Date(end_date).advance(-1, 'month'), end_date).median()

            # Perform change detection
            def detect_change(current, previous):
                previous_img = ee.Image(previous)
                diff = current.select(['B4', 'B3', 'B2']).subtract(previous_img.select(['B4', 'B3', 'B2'])).abs()
                return previous_img.addBands(diff.reduce(ee.Reducer.sum()).rename('change'))  # Return the modified image

            first = s2_collection.sort('system:time_start').first()

            changes_obj = ee.List(s2_collection.sort('system:time_start').iterate(detect_change, first))

            print(changes_obj)

            # changes_obj = s2_collection.sort('system:time_start').iterate(detect_change, first)
            changes = ee.ImageCollection.fromImages(changes_obj)

            # changes = s2_collection.sort('system:time_start').map(lambda image: detect_change(image, first))

            # changes = ee.ImageCollection(s2_collection.sort('system:time_start').iterate(detect_change, first))

            change_image = changes.select('change').max()

            print('Change image:', change_image.getInfo())

            # Apply threshold to identify significant changes
            threshold = 0.01  # Adjust this value based on your needs
            significant_changes = change_image.gt(threshold)


            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson
            }

            return response

print(detect_changes())

ee.List({
  "functionInvocationValue": {
    "functionName": "Collection.iterate",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.limit",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "ImageCollection.load",
                                  "arguments": {
 

EEException: ImageCollection.fromImages: Parameter 'images' is required.

In [53]:


def detect_changes2():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get the closest images
            # start_date = ee.Date('2024-09-01')  # Replace with your start date
            # end_date = ee.Date('2024-09-28')  # Replace with your end date
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Subset bands before calculation (similar to the example you provided)
            s2_start_bands = s2_start.select(['B4', 'B3', 'B2'])
            s2_end_bands = s2_end.select(['B4', 'B3', 'B2'])

            # Calculate difference image
            diff = s2_end_bands.subtract(s2_start_bands).abs() # abs() after the subtraction
            change_image = diff.reduce(ee.Reducer.sum()).rename('change')

            print('Change image:', change_image.getInfo())

            # Apply threshold to identify significant changes
            threshold = 0.6  # Adjust this value based on your needs
            significant_changes = change_image.gt(threshold)

            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response
ch = detect_changes2()


Change image: {'type': 'Image', 'bands': [{'id': 'change', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 600000, 0, -10, 2500020]}]}
Significant changes count: 1


In [54]:
geemap.show_image(ch['start_image'])
geemap.show_image(ch['end_image'])
geemap.show_image(ch['change_image'])

Output()

Output()

Output()

In [56]:
!pip install geojson

In [61]:

import geojson
import folium
import geemap
from IPython.display import display

def detect_changes5():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get the closest images
            # start_date = ee.Date('2024-09-01')  # Replace with your start date
            # end_date = ee.Date('2024-09-28')  # Replace with your end date
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            image1 = images[0]
            image2 = images[1]

            start_image_date = image1.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = image2.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Calculate NDVI for both images
            ndvi1 = image1.normalizedDifference(['B8', 'B4'])
            ndvi2 = image2.normalizedDifference(['B8', 'B4'])

            # Compute NDVI difference
            ndvi_diff = ndvi2.subtract(ndvi1)

            # Threshold the difference image
            threshold = ndvi_diff.abs().gt(0.2)  # Adjust threshold as needed

            # Apply morphological operations to remove noise
            kernel = ee.Kernel.circle(radius=1)
            cleaned = threshold.focal_min(kernel=kernel).focal_max(kernel=kernel)

            # Vectorize the change areas
            vectors = cleaned.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                eightConnected=False,
                maxPixels=1e9,
                geometryType='polygon'
            )

            # Convert to GeoJSON
            geojson_data = vectors.getInfo()

            # Save as GeoJSON file
            with open('change_detection.geojson', 'w') as f:
                geojson.dump(geojson_data, f)

            # Calculate map center manually
            bounds = aoi_geo.bounds().getInfo()['coordinates'][0]
            map_center = [
                (bounds[0][1] + bounds[2][1]) / 2,  # Latitude
                (bounds[0][0] + bounds[1][0]) / 2   # Longitude
            ]

            # Visualize results
            m = folium.Map(location=map_center, zoom_start=10)
            folium.GeoJson(geojson_data).add_to(m)

            return m, start_image_date, end_image_date, geojson_data

# Run the change detection
map_result, start_date, end_date, geojson_data = detect_changes5()

# Display the map
display(map_result)


ch = detect_changes5()


In [ ]:
def detect_changes_ccd():
    if True:
        if True:
            # Parse input parameters
            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()

            # Load Sentinel-1 imagery (using VV polarization for coherence)
            s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                .filterBounds(aoi_geo) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

            # Get the closest images
            images = get_closest_images(s1_collection, start_date, end_date)
            s1_start = images[0]
            s1_end = images[1]

            start_image_date = s1_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s1_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Calculate coherence
            coherence = ee.Image.coherence(s1_start, s1_end)

            # Threshold coherence to identify changes
            threshold = 0.5
            significant_changes = coherence.lt(threshold)

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs (using S1 for visualization)
            start_image_url = s1_start.getThumbURL({
                'min': -25,
                'max': 0,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': 'VV'
            })

            end_image_url = s1_end.getThumbURL({
                'min': -25,
                'max': 0,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': 'VV'
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response

ch_ccd = detect_changes_ccd()

In [ ]:

for k in ch.keys():
    if(k == 'changes_geojson'):
        print(f"'{k}' : {ch[k]},")
    else:
        print(f"'{k}' : '{ch[k]}',")

'start_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/d9ca3650cc9bb81f8dfa00e199488d84-75c293210111bdebdc76b67a5f40f8a2:getPixels',
'end_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3bbec56d91c3f3acb5b4883bde516121-21b2ef1a6e0f5f87181965f3ee10e8ff:getPixels',
'change_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/938736a42876bfcd98b4216344922204-458b48be4ba055f051bca5b9b3b07936:getPixels',
'changes_geojson' : {'type': 'FeatureCollection', 'columns': {'count': 'Long<0, 4294967295>', 'label': 'Byte<0, 1>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[82.13436931633424, 22.189324478416093], [82.13446630616885, 22.18932380290995], [82.13445760192494, 22.188239893352513], [82.14182876238527, 22.18818838863434], [82.14182949242813, 22.18827871420921], [82.15424403729492, 22.18819121539898]

In [ ]:
# show images start end and chnages

geemap.show_image(ch['start_image'])
geemap.show_image(ch['end_image'])
geemap.show_image(ch['change_image'])

Output()

Output()

Output()

In [ ]:
def detect_changes3():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            classifier = ee.Classifier.load('USGS/NLCD/NLCD2016')

            # Get closest non-cloudy Sentinel-2 images
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                            .filterBounds(aoi_geo) \
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Classify start and end images
            s2_start_classified = images[0].classify(classifier)
            s2_end_classified = images[1].classify(classifier)

            # Identify changed pixels based on classification difference
            change_mask = s2_start_classified.neq(s2_end_classified)

             # Convert change mask to polygons
            change_polygons = change_mask.reduceToVectors(
                geometry=aoi_geo,
                scale=30,  # Adjust scale as needed
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = change_mask.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(change_polygons)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response



In [ ]:
c3 = detect_changes3()



geemap.show_image(c3['start_image'])
geemap.show_image(c3['end_image'])
geemap.show_image(c3['change_image'])

EEException: Asset 'USGS/NLCD/NLCD2016' is not a Classifier.

In [ ]:


def detect_changes4():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get the closest images
            # start_date = ee.Date('2024-09-01')  # Replace with your start date
            # end_date = ee.Date('2024-09-28')  # Replace with your end date
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Cloud masking (example using a simple threshold)
            cloudMask = s2_start.select('QA60').bitwiseAnd(1 << 4).eq(0)
            s2_start = s2_start.updateMask(cloudMask)
            s2_end = s2_end.updateMask(cloudMask)

            # Normalization (example using min-max scaling)
            minVal = ee.Number(0)
            maxVal = ee.Number(10000)
            s2_start = s2_start.divide(maxVal).subtract(minVal).multiply(255).uint8()
            s2_end = s2_end.divide(maxVal).subtract(minVal).multiply(255).uint8()


            # Calculate difference image
            diff = s2_end.subtract(s2_start).abs() # abs() after the subtraction
            # change_image = diff.reduce(ee.Reducer.sum()).rename('change')


            # Apply threshold to identify significant changes
            threshold = 0.6  # Adjust this value based on your needs
            significant_changes = diff.gt(threshold)

            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Visualization (example using a colormap)
            colormap = ['000000', 'FF0000', '00FF00']  # Black, Red, Green
            change_image = diff.updateMask(significant_changes).visualize({
                'min': -threshold,
                'max': threshold,
                'palette': colormap
            })
            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response
ch4 = detect_changes4()




Significant changes count: 26


EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [ ]:
print(ch4)

geemap.show_image(ch4['start_image'])
geemap.show_image(ch4['end_image'])
geemap.show_image(ch4['change_image'])

In [15]:
def get_closest_images_ccd(s1_collection, start_date, end_date):
    # return A list of 2 images containing the closest non-None images BEFORE the start and end dates.
    start_images = s1_collection.filterDate('1900-01-01', start_date).sort('system:time_start',False)
    end_images = s1_collection.filterDate('1900-01-01', end_date).sort('system:time_start',False)

    # get the first non none image
    start_img = None
    end_img = None

    # Iterate to find the first non-None image for the start date
    start_list = start_images.toList(start_images.size()) # convert the collection to a list
    for i in range(start_images.size().getInfo()):
        img = ee.Image(start_list.get(i)) # get the image from the list using index i
        if img.getInfo() is not None:
            start_img = img
            break

    # Iterate to find the first non-None image for the end date
    end_list = end_images.toList(end_images.size())
    for i in range(end_images.size().getInfo()):
        img = ee.Image(end_list.get(i))
        if img.getInfo() is not None:
            end_img = img
            break

    return [start_img, end_img]


In [34]:
def detect_changes_ccd():
    if True:
        if True:
            # Parse input parameters
            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2023-06-01"
            end_date = "2024-05-08"
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()

            # Load Sentinel-1 imagery (using VV polarization for coherence)
            s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            # print(f'{s1_collection.size().getInfo() = }')
            if s1_collection.size().getInfo() == 0:
                print("No Sentinel-1 imagery found for the specified dates and region.")
                return None

            # Get the closest images
            images = get_closest_images_ccd(s1_collection, start_date, end_date)
            s1_start = images[0]
            s1_end = images[1]

            start_image_date = s1_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s1_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            print(f'{start_image_date = }')
            print(f'{end_image_date = }')

            print(f'{type(s1_start) = }')
            print(f'{type(s1_end) = }')

            print("Get Info")
            print(f'{s1_start.getInfo() = }')
            print(f'{s1_end.getInfo() = }')

            # Calculate coherence
            coherence = ee.Image.variance(s1_start, s1_end)

            # Threshold coherence to identify changes
            threshold = 0.5
            significant_changes = coherence.lt(threshold)

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs (using S1 for visualization)
            start_image_url = s1_start.getThumbURL({
                'min': -25,
                'max': 0,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': 'VV'
            })

            end_image_url = s1_end.getThumbURL({
                'min': -25,
                'max': 0,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': 'VV'
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response

ch_ccd = detect_changes_ccd()

start_image_date = '2023-05-29 00:21:20'
end_image_date = '2024-04-29 00:21:23'
type(s1_start) = <class 'ee.image.Image'>
type(s1_end) = <class 'ee.image.Image'>
Get Info
s1_start.getInfo() = {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [28550, 21879], 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 613261.1218488427, 0, -10, 2589808.9673147397]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [28550, 21879], 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 613261.1218488427, 0, -10, 2589808.9673147397]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [21, 10], 'crs': 'EPSG:32644', 'crs_transform': [-12658.860430433764, -3303.655031815055, 894519.0615365268, 2127.6003053602763, -20210.643526479602, 2542921.4717027587]}], 'version': 1727677363269847, 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230529T002120_20230529T002145_048741_05DCA2_B3A2', '

AttributeError: type object 'Image' has no attribute 'variance'

In [ ]:
# Coherent Change Detection
print(ch_ccd)

geemap.show_image(ch_ccd['start_image'])
geemap.show_image(ch_ccd['end_image'])
geemap.show_image(ch_ccd['change_image'])

In [28]:
# Define your region of interest (replace with your actual coordinates)
aoi = ee.Geometry.Polygon(
        [[[8.473892211914062, 49.98081240937428],
          [8.658599853515625, 49.98081240937428],
          [8.658599853515625, 50.06066538593667],
          [8.473892211914062, 50.06066538593667],
          [8.473892211914062, 49.98081240937428]]])

# Dates for your two images (replace with your actual dates)
date1 = '2020-08-01'
date2 = '2020-08-15'

# Get the two images from the COPERNICUS/S1_GRD collection
image1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(date1, ee.Date(date1).advance(1, 'day')) \
    .first()

image2 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(date2, ee.Date(date2).advance(1, 'day')) \
    .first()

if image1.size().getInfo() == 0 or image2.size().getInfo() == 0:
  print("No images found for the specified date range and AOI.")

# Select the VV band from both images
vv1 = image1.select('VV')
vv2 = image2.select('VV')

# Calculate the difference between the two images
diff = vv2.subtract(vv1)

# Calculate the variance of the difference image
variance = diff.reduceRegion(
  reducer= ee.Reducer.variance(),
  geometry= aoi,
  scale= 10 # Adjust scale if necessary
).get('VV')

# Print the variance
print(variance.getInfo())

# Display the variance image
# Import the Folium library.
import folium
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[50.020, 8.560], zoom_start=10)

# Add the variance layer to the map
my_map.add_ee_layer(variance, {'min': 0, 'max': 100}, 'Variance')

# Display the map.
display(my_map)

AttributeError: 'Image' object has no attribute 'size'

In [35]:
!pip install  geojsonio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for geojsonio: filename=geojsonio-0.0.3-py3-none-any.whl size=5022 sha256=4bfca11df7384fd26c63060e395e39b7cdb6088dff5d5859acf604aa2019881c
  Stored in directory: /root/.cache/pip/wheels/5b/53/d5/01006178d29712c91b7917f736192c5bb5fcf6e4a06816b903
Successfully built geojsonio


In [43]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 64.2 MB/s eta 0:00:00


In [50]:
# import ee
import cv2
import numpy as np
from skimage.morphology import opening, closing
import geojsonio
import geemap
import rasterio

def detect_changes_gee(image1, image2, threshold=0.1):
    # Preprocessing
    # Ensure images are in grayscale and normalized
    gray1 = image1.select('VV').multiply(10000).cast('uint8')
    gray2 = image2.select('VV').multiply(10000).cast('uint8')

    # Download images as GeoTIFF files (uncomment if preferred)
    task1 = ee.batch.Export.image(image=gray1)
    task2 = ee.batch.Export.image(image=gray2)

    # Start the tasks and retrieve the download URLs
    task1.start()
    task2.start()
    url1 = task1.status()['downloadUrls'][0]
    url2 = task2.status()['downloadUrls'][0]

    # Wait for the downloads to complete (optional)
    ee.batch.Task.wait_for_all(task_ids=[task1.id, task2.id])

    # Load the downloaded GeoTIFF files
    with rasterio.open(url1) as src:
        np_gray1 = src.read(1).astype(np.uint8)  # Read the first band (VV)
    with rasterio.open(url2) as src:
        np_gray2 = src.read(1).astype(np.uint8)  # Read the first band (VV)

    # Download images directly to NumPy arrays (Option 2: Direct download)
    # np_gray1 = ee.Image(gray1).download(scale=10, projection='EPSG:4326').select('VV').array().astype(np.uint8)
    # np_gray2 = ee.Image(gray2).download(scale=10, projection='EPSG:4326').select('VV').array().astype(np.uint8)

    # Change detection (pixel-based difference)
    change_map = np.abs(np_gray1 - np_gray2)

    # Post-processing
    change_map = opening(change_map)
    change_map = closing(change_map)
    change_map = (change_map > threshold).astype(np.uint8) * 255

    # Vectorization
    contours, _ = cv2.findContours(change_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = [polygon.tolist() for polygon in contours]
    geojson_data = {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": polygon}} for polygon in polygons]}

    # Visualization (optional)
    # Create an EE image from the change map
    change_image = ee.Image.fromByte(change_map)
    # ... (add code to display the change image using GEE's visualization tools)

    return change_image, geojson_data

In [51]:

aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
start_date =  "2023-06-01"
end_date = "2024-05-08"
aoi_geo = geemap.geojson_to_ee(aoi).geometry()

# Load Sentinel-1 imagery (using VV polarization for coherence)
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi_geo) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
# print(f'{s1_collection.size().getInfo() = }')
if s1_collection.size().getInfo() == 0:
    print("No Sentinel-1 imagery found for the specified dates and region.")

# Get the closest images
images = get_closest_images_ccd(s1_collection, start_date, end_date)
s1_start = images[0]
s1_end = images[1]

start_image_date = s1_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
end_image_date = s1_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

print(f'{start_image_date = }')
print(f'{end_image_date = }')

print(f'{type(s1_start) = }')
print(f'{type(s1_end) = }')

print("Get Info")
print(f'{s1_start.getInfo() = }')
print(f'{s1_end.getInfo() = }')


# Detect changes
change_image, geojson_data = detect_changes_gee(s1_start, s1_end)

map = geemap.Map()
map.addLayer(change_image, {'min': 0, 'max': 1, 'palette': ['black', 'white']}, 'Change Map')
map.centerObject(aoi_geo, 8)  # Adjust zoom level as needed
map.addLayer(s1_start, {'min': -25, 'max': 0, 'bands': 'VV'}, 'Start Image')
map.addLayer(s1_end, {'min': -25, 'max': 0, 'bands': 'VV'}, 'End Image')
map.addLayer(ee.FeatureCollection(geojson_data), {}, 'Changes (GeoJSON)')
map.update_center_on_interaction(True)
map.show()


start_image_date = '2023-05-29 00:21:20'
end_image_date = '2024-04-29 00:21:23'
type(s1_start) = <class 'ee.image.Image'>
type(s1_end) = <class 'ee.image.Image'>
Get Info
s1_start.getInfo() = {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [28550, 21879], 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 613261.1218488427, 0, -10, 2589808.9673147397]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [28550, 21879], 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 613261.1218488427, 0, -10, 2589808.9673147397]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [21, 10], 'crs': 'EPSG:32644', 'crs_transform': [-12658.860430433764, -3303.655031815055, 894519.0615365268, 2127.6003053602763, -20210.643526479602, 2542921.4717027587]}], 'version': 1727680876651011, 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230529T002120_20230529T002145_048741_05DCA2_B3A2', '

KeyError: 'downloadUrls'